In [1]:
from FEnicS_base import *
import dolfinx
from dolfinx import mesh, fem, io, nls
from dolfinx.fem import VectorFunctionSpace
from mpi4py import MPI
import numpy as np
import ufl
import matplotlib.pyplot as plt

import jsonpickle as jp
import shutil
import re
import inspect
from tqdm import tqdm

In [3]:
# domain = mesh.create_interval(nx=100, comm=MPI.COMM_WORLD, points=[0, 1])
domain = mesh.create_unit_square(
    MPI.COMM_WORLD,
    10,
    10,
    cell_type=mesh.CellType.quadrilateral,
)
element = ufl.FiniteElement(family='CG', cell=domain.ufl_cell(), degree=1)
element2 = ufl.VectorElement(family='CG', cell=domain.ufl_cell(), degree=1)
space = FunctionSpace(domain, element=ufl.MixedElement(element, element2))
U = space.sub(0).collapse()[0]
V = space.sub(1).collapse()[0]

x = SpatialCoordinate(space)[0]
y = SpatialCoordinate(space)[1]
set_connectivity(domain)

ds = Measure("ds", domain=domain)
norm = FacetNormal(domain)

func = Function(space)
sub_u, sub_g = func.sub(0), func.sub(1)

u, v = TrialFunctions(space)
fi, psi = TestFunctions(space)

f = Constant(space, 2)

F = (v|inner|grad(fi)) * dx - (f|dot|fi)*dx
F += (v|inner|psi) * dx - (grad(u)|inner|psi) * dx

bcs = [
    #  DirichletBC(
    #      space=(space.sub(1), V),
    #      form=Function(V, vector(1, 1)),
    #      combined_marker=lambda x: np.isclose(x[0], 0)
    #  ),
    #  DirichletBC(
    #      space=(space.sub(1), V),
    #      form=Function(V, vector(1, 1)),
    #      combined_marker=lambda x: np.isclose(x[0], 1)
    #  ),
    DirichletBC(
        space=(space.sub(0), U),
        form=Function(U, 2),
        combined_marker=lambda x: np.isclose(x[0], 0)
    ),
    # DirichletBC(
    #     space=(space.sub(0), U),
    #     form=Function(U, 1),
    #     combined_marker=lambda x: np.isclose(x[0], 1)
    # ),
]
problem = LinearProblem(
    L=rhs(F),
    a=lhs(F),
    bcs=bcs,
)

In [4]:
us = problem.solve()
us.x.array

array([inf, inf, inf, ..., inf, inf, inf])

In [5]:
func_plot2D(*[Function(U, us,'base')])

RuntimeError: Interpolation: elements have different value dimensions